In [82]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64

import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "generic123"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
df = df.drop('_id', 1)
                               

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#options dictionary for 2 radio button rows *key is for first row**list of values is for second*
all_options = {
    'All': ['All'],
    'Animal': ['Dog', 'Cat', 'Other'],
    'Outcome': ['Adoption', 'Transfer', 'Euthanasia', 'Return to Owner', 'Died'],
    'Sex Result': ['Intact Female', 'Intact Male', 'Spayed Female', 'Neutered Male', 'Unknown']
}

#encoding images *logo* *custom picture*
image_filename = 'Logo.png' 
encoded_image_logo = base64.b64encode(open(image_filename, 'rb').read())

image_filename = "dog.jpg" 
encoded_image_dog = base64.b64encode(open(image_filename, "rb").read())

app.layout = html.Div([
     
    html.Div(id="hidden_div", style={"display": "none"}),
    html.A(
        [
        html.Img(
            src="data:image/png;base64,{}".format(
            encoded_image_logo.decode()
                ),
        style={"height": "2" "00px","padding":"5px 5px 5px 250px", "align":"center", "margin-left":"auto"},
            )
        ],
        href="https://www.snhu.edu"),
       
    html.Img(
            src="data:image/png;base64,{}".format(encoded_image_dog.decode()),
            style={"height": "2" "00px","padding":"5px 5px 5px 150px"}
            ), 
    
    html.Div([
        html.H4(
            children="Created by Gary Clark",
            style={"textAlign": "right", "color": "black"}
             )
        ]),
     
            
    html.Center(html.B(html.H1('AAC Database Dashboard'))),
    html.Hr(),
    html.Div([
        html.Div([
            dcc.RadioItems(id='rescue_filters',
                            options =[
                                {'label':'Water', 'value':'WTR'},
                                {'label':'Mtn or Wilderness', 'value':'MTN'},
                                {'label':'Disaster or Individual Tracking', 'value':'DIT'},
                                {'label':'Reset/All', 'value':'RST'}],
                                value = 'RST')
            ], style={'fontSize':14, 'text-align':'center'}),
        html.Hr(),
        
        html.Div([
            dcc.RadioItems(
                id='additional_filters',
                options=[{'label': k, 'value': k} for k in all_options.keys()],
                value='All'
            ),
            html.Hr(),
            dcc.RadioItems(id='additional_filters_radio')
        ], style={'fontSize': 12, 'text-align':'center'}),
    ]),
        
        
          
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        #added hideable attribute to allow user to toggle view of columns
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True} for i in df.columns
        ],
        style_table={'overflowX': 'auto'}, #horizontal scroll bar for table
        #formatting the cells to look less cluttered using overflow
        style_cell={
            'minWidth': '75px',
            'maxWidth': '125px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            
        },
        #enables markdown so if data is cut off by ellipsis user can hover cursor to read full text
        tooltip_data=[
            {
                column: {'value': str(value), 'type': 'markdown'}
                for column, value in row.items()
            } for row in df.to_dict('records')
        ],
        tooltip_duration=None,
        
        data=df.to_dict('records'),
        
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10, 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    ),

                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    )
            ])
     ])



#############################################
# Interaction Between Components / Controller
#############################################



#this callback updates the 2nd radio buttons row with input from 1st row
@app.callback(
    Output('additional_filters_radio', 'options'),
    [Input('additional_filters', 'value')])
def set_cities_options(selected_filter):
    return [{'label': i, 'value': i} for i in all_options[selected_filter]]

#this callback takes the options passed from the first callback and updates the display of 2nd, outputs value for sort
@app.callback(
    Output('additional_filters_radio', 'value'),
    [Input('additional_filters_radio', 'options')])
def set_cities_value(available_options):
    return available_options[0]['value']

#this callback enables the user to filter which results are displayed as radio items using common queries
@app.callback(
    Output('datatable-id', "data"),
    [Input('additional_filters_radio', 'value'),
    Input('rescue_filters', 'value')]
)
def animalSort(value, rescue):
        if value == "Other":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Other"}))
            df = df.drop('_id', 1)
        elif value == "Cat":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Cat"}))
            df = df.drop('_id', 1)
        elif value == "Dog":
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
            df = df.drop('_id', 1)
        elif value == "Adoption":
            df = pd.DataFrame.from_records(shelter.read({"outcome_type":"Adoption"}))
            df = df.drop('_id', 1)
        elif value == "Transfer":
            df = pd.DataFrame.from_records(shelter.read({"outcome_type":"Transfer"}))
            df = df.drop('_id', 1)
        elif value == "Euthanasia":
            df = pd.DataFrame.from_records(shelter.read({"outcome_type":"Euthanasia"}))
            df = df.drop('_id', 1)
        elif value == "Return to Owner":
            df = pd.DataFrame.from_records(shelter.read({"outcome_type":"Return to Owner"}))
            df = df.drop('_id', 1)
        elif value == "Died":
            df = pd.DataFrame.from_records(shelter.read({"outcome_type":"Died"}))
            df = df.drop('_id', 1)
        elif value == "Intact Female":
            df = pd.DataFrame.from_records(shelter.read({"sex_upon_outcome":"Intact Female"}))
            df = df.drop('_id', 1)
        elif value == "Intact Male":
            df = pd.DataFrame.from_records(shelter.read({"sex_upon_outcome":"Intact Male"}))
            df = df.drop('_id', 1)
        elif value == "Neutered Male":
            df = pd.DataFrame.from_records(shelter.read({"sex_upon_outcome":"Neutered Male"}))
            df = df.drop('_id', 1)
        elif value == "Spayed Female":
            df = pd.DataFrame.from_records(shelter.read({"sex_upon_outcome":"Spayed Female"}))
            df = df.drop('_id', 1)
        elif value == "Unknown":
            df = pd.DataFrame.from_records(shelter.read({"sex_upon_outcome":"Unknown"}))
            df = df.drop('_id', 1)
        elif rescue == "WTR":
            df = pd.DataFrame.from_records(shelter.read({'$and':
                                                             [{"sex_upon_outcome":"Intact Male"},
                                                             {'breed':{'$in':["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                                                             {"age_upon_outcome_in_weeks":{'$gte':26, '$lte':156}}
                                                             ]}))
            df = df.drop('_id', 1)
        elif rescue == "MTN":
            df = pd.DataFrame.from_records(shelter.read({'$and':
                                                        [{"breed":{'$in':["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound","Rottweiler"]}}, 
                                                         {"sex_upon_outcome":"Intact Male"}, 
                                                         {"age_upon_outcome_in_weeks":{'$gte':20, '$lte':300}}
                                                        ]}))
            df = df.drop('_id', 1)
        elif rescue == "DIT":
            df = pd.DataFrame.from_records(shelter.read({'$and': 
                                                        [{"breed":{'$in':["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}, 
                                                         {"sex_upon_outcome":"Intact Male"}, 
                                                         {"age_upon_outcome_in_weeks":{'$gte':20, '$lte':156}}
                                                        ]}))
            df = df.drop('_id', 1)
        elif rescue == "RST":
            df = pd.DataFrame.from_records(shelter.read({}))
            df = df.drop('_id', 1)
        else:
            df = pd.DataFrame.from_records(shelter.read({})) #default value is to show unfiltered table
            df = df.drop('_id', 1)
        
        return df.to_dict('records')


#this callback highlights the selected row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(data):
    #enabling pie chart view of breeds
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(dff, names="breed")
    return dcc.Graph(figure=fig)            


#this callback displays geolocation data of selected item
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]





app